Solutions for windows notebook

In [1]:
# Load the extension
%load_ext sql

In [2]:
# Connect to DuckDB
# NOTE: ONLY 1 NOTEBOOK CAN BE CONNECTED TO DUCKDB AT ANY TIME
%sql duckdb:///../../tpch.db

RuntimeError: (duckdb.duckdb.IOException) IO Error: Could not set lock on file "/home/josephkevinmachado/code/adv_data_transformation_in_sql/concepts/windows/../../tpch.db": Conflicting lock is held in /home/josephkevinmachado/.asdf/installs/python/3.12.4/bin/python3.12 (PID 2488660) by user josephkevinmachado. See also https://duckdb.org/docs/connect/concurrency
(Background on this error at: https://sqlalche.me/e/20/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
%%sql
-- USER EXERCISE 1
SELECT
  o_orderdate,
  o_custkey,
  o_totalprice,
  o_totalprice - lag (o_totalprice) OVER (
    PARTITION BY
      o_custkey
    ORDER BY
      o_orderdate
  ) AS totalprice_diff
FROM
  orders
ORDER BY
  o_custkey,
  o_orderdate
LIMIT
  50;

In [ ]:
%%sql
-- USER EXERCISE 2
SELECT
  order_month,
  o_custkey,
  total_price,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        CAST(order_month AS DATE) RANGE BETWEEN INTERVAL '1' MONTH PRECEDING
        AND INTERVAL '1' MONTH FOLLOWING
    ),
    2
  ) AS consecutive_three_mo_total_price_avg,
  ROUND(
    AVG(total_price) OVER (
      PARTITION BY
        o_custkey
      ORDER BY
        order_month ROWS BETWEEN 1 PRECEDING
        AND 1 FOLLOWING
    ),
    2
  ) AS three_mo_total_price_avg
FROM
  (
    SELECT
      strftime (o_orderdate, '%Y-%m-01') AS order_month,
      o_custkey,
      SUM(o_totalprice) AS total_price
    FROM
      orders
    GROUP BY
      1,
      2
  )
ORDER BY
  o_custkey,
  order_month
LIMIT
  50;